In [15]:
import os
from PyPDF2 import PdfReader
import pandas as pd
import re

# Function to extract information using a regular expression pattern
def extract_info(pattern, text):
    match = re.search(pattern, text)
    return match[1] if match else ""

# Folder path containing PDF files
folder_path = "./data"

# Initialize an empty list to store ticket details from all PDF files
all_ticket_details = []

# Iterate over the PDF files in the folder
for filename in os.listdir(folder_path):
    if filename.endswith(".pdf"):
        # Load the PDF file
        pdf_path = os.path.join(folder_path, filename)
        pdf = PdfReader(pdf_path)

        # Extract the text data from the first page
        page = pdf.pages[0]
        extracted_text = page.extract_text()

        # Extract ticket details for the current PDF
        ticket_info = {
            "PNR": extract_info(r"PNR No\. : (\d+)", extracted_text),
            "Train Number": extract_info(
                r"Train No\. / Name : (\d+) / (.+)", extracted_text
            ),
            "Train Name": extract_info(
                r"Train No\. \/ Name : \d+ \/ (.+?) Quota", extracted_text
            ),
            "Quota": extract_info(r"Quota : (.+)", extracted_text),
            "Transaction ID": extract_info(
                r"Transaction ID : (\d+)", extracted_text
            ),
            "Date of Booking": extract_info(
                r'Date & T ime of Booking : (\d{2}-[A-Za-z]{3}-\d{4})',
                extracted_text,
            ),
            "Class": extract_info(r"Class : (.+)", extracted_text),
            "From": re.search(r"From : (.*?) \(.*?\)", extracted_text)[1],
            "Date of Journey": extract_info(
                r"Date of Journey : (\d+-[A-Za-z]+-\d+)", extracted_text
            ),
            "To": re.search(r"To : (.*?) \(.*?\)", extracted_text)[1],
            "Boarding At": extract_info(
                r"Boarding At : (.+)", extracted_text
            ).split(" ")[0],
            "Scheduled Departure": extract_info(
                r"Scheduled Departure\* : (\d+-[A-Za-z]+-\d+ \d+:\d+)",
                extracted_text,
            ),
            "Reservation Up to": extract_info(
                r"Reservation Up to : (.+)", extracted_text
            ).split(" Scheduled Arrival")[0],
            "Scheduled Arrival": extract_info(
                r"Scheduled Arrival : (\d+-[A-Za-z]+-\d+ \d+:\d+)",
                extracted_text,
            ),
            "Adult": extract_info(r"Adult:  (\d+)", extracted_text),
            "Child": extract_info(r"Child:  (\d+)", extracted_text),
            "Passenger Mobile No": extract_info(
                r"Passenger Mobile No : (.+)", extracted_text
            ).split(" Distance")[0],
            "Distance": extract_info(r"Distance : (\d+)KM", extracted_text),
            "Insurance (No. of Psng)": extract_info(
                r"Insurance \(No\. of Psng\) : (\d+)", extracted_text
            ),
        }

        # Extract passenger details
        passenger_details_match = re.findall(r"(\d) (.+?) (\d+) (Female|Male) (\w+) (\w+) (\d+)", extracted_text)
        passenger_details = []
        for match in passenger_details_match:
            passenger = {
                "Passenger": match[0],
                "Name": match[1],
                "Age": match[2],
                "Gender": match[3],
                "Status": match[4],
                "Coach": match[5],
                "Seat / Berth / WL No": match[6]
            }
            passenger_details.append(passenger)

        # Create DataFrame for ticket details
        ticket_df = pd.DataFrame([ticket_info])

        if passenger_details:
            # Create DataFrame for passenger details
            passenger_df = pd.DataFrame(passenger_details)

            # Duplicate ticket details to match the number of passengers
            ticket_df = pd.concat([ticket_df] * len(passenger_df), ignore_index=True)

            # Combine ticket and passenger details into a single DataFrame
            combined_df = pd.concat([ticket_df, passenger_df], axis=1)

            # Extract fare details
            fare_match = re.findall(r"\d+\.\d+", extracted_text)
            total_fare = max(fare_match, key=float) if fare_match else ""

            # Add Total Fare column to the DataFrame
            combined_df["Total Fare"] = total_fare

            # Append ticket details to the list
            all_ticket_details.append(combined_df)

# Combine ticket details from all PDF files into a single DataFrame
all_ticket_details_df = pd.concat(all_ticket_details, ignore_index=True)

# Save the combined DataFrame to an Excel file
output_folder = "./output"
os.makedirs(output_folder, exist_ok=True)
output_path = os.path.join(output_folder, "Ticket_Details.xlsx")
all_ticket_details_df.to_excel(output_path, index=False)

print("Excel file saved successfully!")


Excel file saved successfully!


In [16]:
all_ticket_details_df

,PNR,Train Number,Train Name,Quota,Transaction ID,Date of Booking,Class,From,Date of Journey,To,...,Distance,Insurance (No. of Psng),Passenger,Name,Age,Gender,Status,Coach,Seat / Berth / WL No,Total Fare
0,6420427624,13071,HWH JMP EXPRESS,GENERAL,100004203349628,31-May-2023,THIRD AC,BOLPUR S NIKTN,12-Jul-2023,BHAGALPUR,...,269,1,1,PUJA SINGH,31,Female,CNF,B1,25,540.75
1,6321030997,13015,KA VIGURU EXPRES,GENERAL,100004233521936,13-Jun-2023,SECOND SITTING,RAMPUR HA T,14-Jun-2023,BHAGALPUR,...,181,4,1,PAWAN KUMAR SING,44,Male,CNF,D2,19,379.10
2,6321030997,13015,KA VIGURU EXPRES,GENERAL,100004233521936,13-Jun-2023,SECOND SITTING,RAMPUR HA T,14-Jun-2023,BHAGALPUR,...,181,4,2,SARIKA MAMI,34,Female,CNF,D2,20,379.10
3,6321030997,13015,KA VIGURU EXPRES,GENERAL,100004233521936,13-Jun-2023,SECOND SITTING,RAMPUR HA T,14-Jun-2023,BHAGALPUR,...,181,4,3,ANKIT ANAND,15,Male,CNF,D2,21,379.10
4,6321030997,13015,KA VIGURU EXPRES,GENERAL,100004233521936,13-Jun-2023,SECOND SITTING,RAMPUR HA T,14-Jun-2023,BHAGALPUR,...,181,4,4,AYUSH ANAND,13,Male,CNF,D2,22,379.10
5,6320030680,12041,SHA TABDI EXP,GENERAL,100004184610509,22-May-2023,CHAIR CAR,HOWRAH JN,22-May-2023,BOLPUR S NIKTN,...,146,2,1,PUJA SINGH,31,Female,CNF,C5,44,1026.10
6,6320030680,12041,SHA TABDI EXP,GENERAL,100004184610509,22-May-2023,CHAIR CAR,HOWRAH JN,22-May-2023,BOLPUR S NIKTN,...,146,2,2,PRASHANT KUMAR,29,Male,CNF,C5,43,1026.10
7,6119355827,13242,RJPB BANKA EXP,GENERAL,100004149172713,08-May-2023,THIRD AC,RAJENDRANAGAR T,01-Aug-2023,BHAGALPUR,...,220,2,1,PUJA SINGH,31,Female,CNF,B1,49,1046.10
8,6119355827,13242,RJPB BANKA EXP,GENERAL,100004149172713,08-May-2023,THIRD AC,RAJENDRANAGAR T,01-Aug-2023,BHAGALPUR,...,220,2,2,PUSHP A SINHA,48,Female,CNF,B1,52,1046.10
9,6218029241,22948,BGP SURA T EXP,GENERAL,100004079596078,07-Apr-2023,THIRD AC,BHAGALPUR,24-Jul-2023,PATNA JN,...,222,3,1,PUSHP A SINHA,55,Female,CNF,B1,47,1701.45
